In [59]:
import pickle
import os

with open(os.path.join("..", "data", "runs", "iqm_envs_c3po.pickle"), "rb") as f:
    iqm_scores, iqm_cis = pickle.load(f)

iqm_scores, iqm_cis

({'SafetyAntVelocity-v1': {'Reward': {'C3PO (ours)': array([  94.14362916, 1143.03286142, 1782.62650643, 2244.59333351,
           2439.08288733, 2630.29018628, 2715.0704779 , 2857.27203204,
           2974.47572551, 3043.58323649]),
    'P3O (our version)': array([-102.11329686,   -6.10051112,  303.69008683,  775.48873939,
           1146.54838289, 1579.48128908, 1863.26400494, 2159.68824147,
           2323.98386981, 2438.73258314]),
    'PPO-Lag': array([  44.64752112,  850.46453855, 1740.87764482, 2406.07964188,
           2611.01639871, 2825.70691856, 2972.83608587, 3019.75313962,
           3159.7161256 , 3210.73767024]),
    'P3O': array([  15.65464698,  816.07300382, 1374.60893898, 1931.03623362,
           2252.57892194, 2551.22191307, 2740.63596927, 2952.09051328,
           3085.24772277, 3122.52175845]),
    'PPO': array([  44.64752112,  850.46453855, 1527.74249583, 2495.92760747,
           3160.74882962, 3913.83188112, 4104.40281877, 4667.22905267,
           5037.4327761

In [78]:
from helpers import *
ALGOS = [ 
    #"C3PO (ours)",
    "C3PO (ours)",
    "P3O (our version)",
    #"C-TRPO",
    #"C3PO (ours, step)",
    #"C3PO (ours, medium)",
    #"C3PO (ours, large)"
    #"CPO",
    "P3O",
    "PPO-Lag",
    # "P3O (ours, large)",
    "PPO",
]

new_eval_score_dict = {}
for env in ENVS:
    for algo in ALGOS:
        new_eval_score_dict[algo.upper()] = {}
        for metric in ["Reward", "Final Cost"]:
            metric_name = metric #legacy
            new_eval_score_dict[algo.upper()][metric_name] = {}
            for env in ENVS:
                mean = round(iqm_scores[env][metric][algo][-1], 1)
                ci_up = round(iqm_cis[env][metric][algo][0,-1] - mean, 1)
                ci_down = round(iqm_cis[env][metric][algo][1,-1] - mean, 1)
                ci = max(ci_up, ci_down)
                mean, ci = (mean, ci) if mean < 100 else (int(mean), int(ci))
                new_eval_score_dict[algo.upper()][metric_name][env.replace("Safety", "").split("-")[0].strip("1")] = f"{mean} ± {ci}"

latex_lookup = {
    "Reward": r'$R(\pi_\textrm{final})$', "Final Cost": r'$C(\pi_\textrm{final})$'
}

df_eval = pd.DataFrame.from_dict({(i, latex_lookup[j]): new_eval_score_dict[i][j] 
                           for i in new_eval_score_dict.keys() 
                           for j in new_eval_score_dict[i].keys()},
                           orient='index')

df_eval

AntVelocity HalfCheetahVelocity  \
C3PO (OURS)       $R(\pi_\textrm{final})$   3043 ± 44          2458 ± 436   
                  $C(\pi_\textrm{final})$  15.0 ± 4.7          13.3 ± 6.4   
P3O (OUR VERSION) $R(\pi_\textrm{final})$   2438 ± 19          2044 ± 186   
                  $C(\pi_\textrm{final})$  11.7 ± 1.0           7.8 ± 0.8   
P3O               $R(\pi_\textrm{final})$   3122 ± 24           3020 ± 12   
                  $C(\pi_\textrm{final})$  21.2 ± 2.5          27.0 ± 1.1   
PPO-LAG           $R(\pi_\textrm{final})$   3210 ± 85            3033 ± 1   
                  $C(\pi_\textrm{final})$  28.9 ± 8.7          23.2 ± 1.9   
PPO               $R(\pi_\textrm{final})$  5402 ± 274          6583 ± 954   
                  $C(\pi_\textrm{final})$    887 ± 27             976 ± 1   

                                          HumanoidVelocity HopperVelocity  \
C3PO (OURS)       $R(\pi_\textrm{final})$        5389 ± 93      1674 ± 35   
                  $C(\pi_\textrm{final})$        1.2 ± 0.9      9.9 ± 1.7   
P3O (OUR VERSION) $R(\pi_\textrm{final})$       5108 ± 231      1474 ± 42   
                  $C(\pi_\textrm{final})$        3.4 ± 1.1      4.4 ± 4.8   
P3O               $R(\pi_\textrm{final})$       5492 ± 118      1633 ± 49   
                  $C(\pi_\textrm{final})$        4.2 ± 2.2     14.6 ± 1.6   
PPO-LAG           $R(\pi_\textrm{final})$       5814 ± 122      240 ± 159   
                  $C(\pi_\textrm{final})$      12.7 ± 31.0    38.8 ± 36.4   
PPO               $R(\pi_\textrm{final})$       6138 ± 699     1810 ± 390   
                  $C(\pi_\textrm{final})$         783 ± 60       435 ± 85   

                                             CarButton    PointGoal  \
C3PO (OURS)       $R(\pi_\textrm{final})$    2.3 ± 0.7   23.8 ± 0.9   
                  $C(\pi_\textrm{final})$  53.4 ± 22.3   37.9 ± 1.7   
P3O (OUR VERSION) $R(\pi_\textrm{final})$    0.9 ± 0.3   20.5 ± 1.3   
                  $C(\pi_\textrm{final})$   26.9 ± 9.2   16.8 ± 6.0   
P3O               $R(\pi_\textrm{final})$    0.2 ± 0.3    5.7 ± 0.3   
                  $C(\pi_\textrm{final})$  40.9 ± 18.2   17.1 ± 6.2   
PPO-LAG           $R(\pi_\textrm{final})$    0.3 ± 0.8    9.4 ± 1.8   
                  $C(\pi_\textrm{final})$  39.2 ± 41.1  22.5 ± 10.1   
PPO               $R(\pi_\textrm{final})$   18.2 ± 1.2   26.6 ± 0.2   
                  $C(\pi_\textrm{final})$     378 ± 18   50.7 ± 3.3   

                                          RacecarCircle    PointPush  
C3PO (OURS)       $R(\pi_\textrm{final})$    25.9 ± 5.1    4.5 ± 2.6  
                  $C(\pi_\textrm{final})$     5.0 ± 1.7  20.2 ± 10.0  
P3O (OUR VERSION) $R(\pi_\textrm{final})$    20.5 ± 3.1    0.7 ± 0.8  
                  $C(\pi_\textrm{final})$     6.3 ± 1.5   10.2 ± 5.8  
P3O               $R(\pi_\textrm{final})$     0.9 ± 0.1    0.7 ± 0.6  
                  $C(\pi_\textrm{final})$    13.1 ± 4.6   14.1 ± 9.4  
PPO-LAG           $R(\pi_\textrm{final})$    30.9 ± 1.8    0.6 ± 0.0  
                  $C(\pi_\textrm{final})$    31.7 ± 2.7   18.2 ± 9.5  
PPO               $R(\pi_\textrm{final})$    40.8 ± 0.5    0.9 ± 0.7  
                  $C(\pi_\textrm{final})$       200 ± 4  42.9 ± 24.0

In [79]:
ids = pd.IndexSlice
admissible = {
    env: list(df_eval[env].loc[ids[:, r"$C(\pi_\textrm{final})$"]].apply(lambda x: float(x.split(" ± ")[0]))[df_eval[env].loc[ids[:, r"$C(\pi_\textrm{final})$"]].apply(lambda x: float(x.split(" ± ")[0]) + float(x.split(" ± ")[1])) < 25.0].index) for env in df_eval.columns
}
admissible

{'AntVelocity': ['C3PO (OURS)', 'P3O (OUR VERSION)', 'P3O'],
 'HalfCheetahVelocity': ['C3PO (OURS)', 'P3O (OUR VERSION)'],
 'HumanoidVelocity': ['C3PO (OURS)', 'P3O (OUR VERSION)', 'P3O'],
 'HopperVelocity': ['C3PO (OURS)', 'P3O (OUR VERSION)', 'P3O'],
 'CarButton': [],
 'PointGoal': ['P3O (OUR VERSION)', 'P3O'],
 'RacecarCircle': ['C3PO (OURS)', 'P3O (OUR VERSION)', 'P3O'],
 'PointPush': ['P3O (OUR VERSION)', 'P3O']}

In [80]:
from functools import partial

def bold_formatter(x, value):
    xn = round(float(x.split(" ± ")[0]), 1)
    if xn == value:
        return f"{{\\bfseries {x}}}"
    else:
        return f"{{{x}}}"

def underline_formatter(x, value):
    xn = round(float(x.split(" ± ")[0]), 1)
    if xn == value:
        return f"{{\\fbox{{{x}}}}}"
    else:
        return f"{{{x}}}"

ids = pd.IndexSlice

fmts1 = {column: partial(bold_formatter, value=df_eval[column].loc[ids[admissible[column], r"$R(\pi_\textrm{final})$"]].apply(lambda x: round(float(x.split(" ± ")[0]), 1)).max()) for column in df_eval.columns}

styled = df_eval[["AntVelocity", "HalfCheetahVelocity",	"HumanoidVelocity",	"HopperVelocity"]].style.format(subset=ids[:, r"$R(\pi_\textrm{final})$", :], formatter=fmts1)

styled

In [81]:
print(styled.to_latex(position_float="centering", hrules=True, multirow_align="t", multicol_align="r", clines="skip-last;data",))

\begin{table}
\centering
\begin{tabular}{llllll}
\toprule
 &  & AntVelocity & HalfCheetahVelocity & HumanoidVelocity & HopperVelocity \\
\midrule
\multirow[t]{2}{*}{C3PO (OURS)} & $R(\pi_\textrm{final})$ & {3043 ± 44} & {\bfseries 2458 ± 436} & {5389 ± 93} & {\bfseries 1674 ± 35} \\
 & $C(\pi_\textrm{final})$ & 15.0 ± 4.7 & 13.3 ± 6.4 & 1.2 ± 0.9 & 9.9 ± 1.7 \\
\cline{1-6}
\multirow[t]{2}{*}{P3O (OUR VERSION)} & $R(\pi_\textrm{final})$ & {2438 ± 19} & {2044 ± 186} & {5108 ± 231} & {1474 ± 42} \\
 & $C(\pi_\textrm{final})$ & 11.7 ± 1.0 & 7.8 ± 0.8 & 3.4 ± 1.1 & 4.4 ± 4.8 \\
\cline{1-6}
\multirow[t]{2}{*}{P3O} & $R(\pi_\textrm{final})$ & {\bfseries 3122 ± 24} & {3020 ± 12} & {\bfseries 5492 ± 118} & {1633 ± 49} \\
 & $C(\pi_\textrm{final})$ & 21.2 ± 2.5 & 27.0 ± 1.1 & 4.2 ± 2.2 & 14.6 ± 1.6 \\
\cline{1-6}
\multirow[t]{2}{*}{PPO-LAG} & $R(\pi_\textrm{final})$ & {3210 ± 85} & {3033 ± 1} & {5814 ± 122} & {240 ± 159} \\
 & $C(\pi_\textrm{final})$ & 28.9 ± 8.7 & 23.2 ± 1.9 & 12.7 ± 31.0 & 38.

In [82]:
from functools import partial

def bold_formatter(x, value):
    xn = round(float(x.split(" ± ")[0]), 1)
    if xn == value:
        return f"{{\\bfseries{x}}}"
    else:
        return f"{{{x}}}"

def underline_formatter(x, value):
    xn = round(float(x.split(" ± ")[0]), 1)
    if xn == value:
        return f"{{\\fbox{{{x}}}}}"
    else:
        return f"{{{x}}}"

ids = pd.IndexSlice

fmts1 = {column: partial(bold_formatter, value=df_eval[column].loc[ids[admissible[column], r"$R(\pi_\textrm{final})$"]].apply(lambda x: round(float(x.split(" ± ")[0]), 1)).max()) for column in df_eval.columns}

styled = df_eval[["PointGoal", "PointPush", "CarButton", "RacecarCircle"]].style.format(subset=ids[:, r"$R(\pi_\textrm{final})$", :], formatter=fmts1)

styled

In [83]:
print(styled.to_latex(position_float="centering", hrules=True, multirow_align="t", multicol_align="r", clines="skip-last;data",))

\begin{table}
\centering
\begin{tabular}{llllll}
\toprule
 &  & PointGoal & PointPush & CarButton & RacecarCircle \\
\midrule
\multirow[t]{2}{*}{C3PO (OURS)} & $R(\pi_\textrm{final})$ & {23.8 ± 0.9} & {4.5 ± 2.6} & {2.3 ± 0.7} & {\bfseries25.9 ± 5.1} \\
 & $C(\pi_\textrm{final})$ & 37.9 ± 1.7 & 20.2 ± 10.0 & 53.4 ± 22.3 & 5.0 ± 1.7 \\
\cline{1-6}
\multirow[t]{2}{*}{P3O (OUR VERSION)} & $R(\pi_\textrm{final})$ & {\bfseries20.5 ± 1.3} & {\bfseries0.7 ± 0.8} & {0.9 ± 0.3} & {20.5 ± 3.1} \\
 & $C(\pi_\textrm{final})$ & 16.8 ± 6.0 & 10.2 ± 5.8 & 26.9 ± 9.2 & 6.3 ± 1.5 \\
\cline{1-6}
\multirow[t]{2}{*}{P3O} & $R(\pi_\textrm{final})$ & {5.7 ± 0.3} & {\bfseries0.7 ± 0.6} & {0.2 ± 0.3} & {0.9 ± 0.1} \\
 & $C(\pi_\textrm{final})$ & 17.1 ± 6.2 & 14.1 ± 9.4 & 40.9 ± 18.2 & 13.1 ± 4.6 \\
\cline{1-6}
\multirow[t]{2}{*}{PPO-LAG} & $R(\pi_\textrm{final})$ & {9.4 ± 1.8} & {0.6 ± 0.0} & {0.3 ± 0.8} & {30.9 ± 1.8} \\
 & $C(\pi_\textrm{final})$ & 22.5 ± 10.1 & 18.2 ± 9.5 & 39.2 ± 41.1 & 31.7 ± 2.7 \\
\cli